In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from scipy.sparse import save_npz, load_npz
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

### Load and prepare data

In [2]:
DATA_DIR = os.path.join('..', 'data', 'processed', 'sparse')

In [3]:
train_ratings = load_npz(os.path.join(DATA_DIR, 'train_ratings.npz'))
test_ratings = load_npz(os.path.join(DATA_DIR, 'test_ratings.npz'))

In [4]:
M, N = train_ratings.shape
M, N

(9166, 1998)

In [5]:
# create mask of locations where sparse matrices have nonzero values
train_mask = (train_ratings > 0).astype(np.uint8)
test_mask = (test_ratings > 0).astype(np.uint8)

In [6]:
# compute global mean
mean = train_ratings.sum() / train_mask.sum()
mean

4.2368585786572535

### Build the model

In [7]:
def build_model(M, N, K, lmbda=0.):
    """
    Build a model for matrix factorization
    Args:
        M (int): number of users
        N (int): number of items
        K (int): size of hidden layer
        lmbda (float): L2-regularization parameter
    Returns:
        keras.models.Model: model for MF
    """
    user_vector = Input((N, )) 
    
    noisy_user_vector = Dropout(rate=0.8)(user_vector)
    hidden = Dense(K, activation='relu', kernel_regularizer=l2(lmbda))(noisy_user_vector)
    output = Dense(N, kernel_regularizer=l2(lmbda))(hidden)
    
    model = Model(user_vector, output)
    
    return model

### Example of keras-model architecture

In [8]:
model = build_model(M, N, K=900, lmbda=0.001)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1998)]            0         
_________________________________________________________________
dropout (Dropout)            (None, 1998)              0         
_________________________________________________________________
dense (Dense)                (None, 900)               1799100   
_________________________________________________________________
dense_1 (Dense)              (None, 1998)              1800198   
Total params: 3,599,298
Trainable params: 3,599,298
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [10]:
def sparse_mse(y_true, y_pred):
    true_mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    sq_residuals = ((y_pred - y_true) ** 2) * true_mask # include only residual from non zero inputs
    return tf.reduce_sum(sq_residuals) / tf.reduce_sum(true_mask)

def sparse_rmse(y_true, y_pred):
    return tf.sqrt(sparse_mse(y_true, y_pred))

#### Define data generators

In [11]:
def train_datagen(train_ratings, train_mask, batch_size=128):
    while True:
        train_ratings, train_mask = shuffle(train_ratings, train_mask)
        steps = train_ratings.shape[0] // batch_size
        for step in range(steps):
            train_batch = train_ratings[step * batch_size:(step + 1) * batch_size].toarray()
            mask_batch = train_mask[step * batch_size:(step + 1) * batch_size].toarray()  
            # normilize nonzero values
            train_batch = train_batch - mean * mask_batch
            yield train_batch, train_batch

def test_datagen(train_ratings, train_mask, test_ratings, test_mask, batch_size=128):
    while True:
        steps = train_ratings.shape[0] // batch_size
        for step in range(steps):
            train_batch = train_ratings[step * batch_size:(step + 1) * batch_size].toarray()
            mask_batch = train_mask[step * batch_size:(step + 1) * batch_size].toarray() 
            test_batch = test_ratings[step * batch_size:(step + 1) * batch_size].toarray()
            test_mask_batch = test_mask[step * batch_size:(step + 1) * batch_size].toarray() 
            # normilize nonzero values
            train_batch = train_batch - mean * mask_batch
            test_batch = test_batch - mean * test_mask_batch
            
            yield train_batch, test_batch

In [12]:
LOSS = sparse_mse
LR = 0.1
RATE_DECAY = 0
EPOCHS = 100
BATCH_SIZE = 256

In [13]:
model.compile(
    loss=LOSS,
    optimizer=SGD(lr=LR, decay=RATE_DECAY, momentum=0.9),
    metrics=[sparse_rmse]
)

In [108]:
history = model.fit_generator(
    train_datagen(train_ratings, train_mask, batch_size=BATCH_SIZE),
    validation_data=test_datagen(train_ratings, train_mask, test_ratings, test_mask, batch_size=BATCH_SIZE),
    steps_per_epoch=train_ratings.shape[0] // BATCH_SIZE,
    validation_steps=test_ratings.shape[0] // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/100
35/35 [==============================] - 4s 107ms/step - loss: 3.3584 - sparse_rmse: 0.9879 - val_loss: 3.1887 - val_sparse_rmse: 0.9734
Epoch 2/100
35/35 [==============================] - 4s 110ms/step - loss: 2.9766 - sparse_rmse: 0.9401 - val_loss: 2.8814 - val_sparse_rmse: 0.9626
Epoch 3/100
35/35 [==============================] - 4s 109ms/step - loss: 2.6626 - sparse_rmse: 0.9150 - val_loss: 2.6188 - val_sparse_rmse: 0.9550
Epoch 4/100
35/35 [==============================] - 4s 109ms/step - loss: 2.3978 - sparse_rmse: 0.8964 - val_loss: 2.3924 - val_sparse_rmse: 0.9482
Epoch 5/100
35/35 [==============================] - 4s 109ms/step - loss: 2.1790 - sparse_rmse: 0.8854 - val_loss: 2.1964 - val_sparse_rmse: 0.9420
Epoch 6/100
35/35 [==============================] - 4s 110ms/step - loss: 1.9906 - sparse_rmse: 0.8761 - val_loss: 2.0261 - val_sparse_rmse: 0.9362
Epoch 7/100
35/35 [==============================] - 4s 109ms/step - loss: 1.8236 - sparse_rmse: 0.8653 - 

In [14]:
history = model.fit_generator(
    train_datagen(train_ratings, train_mask, batch_size=BATCH_SIZE),
    validation_data=test_datagen(train_ratings, train_mask, test_ratings, test_mask, batch_size=BATCH_SIZE),
    steps_per_epoch=train_ratings.shape[0] // BATCH_SIZE,
    validation_steps=test_ratings.shape[0] // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/100
35/35 [==============================] - 4s 100ms/step - loss: 3.3559 - sparse_rmse: 0.9871 - val_loss: 3.1882 - val_sparse_rmse: 0.9734
Epoch 2/100
35/35 [==============================] - 3s 97ms/step - loss: 2.9747 - sparse_rmse: 0.9392 - val_loss: 2.8814 - val_sparse_rmse: 0.9628
Epoch 3/100
35/35 [==============================] - 3s 98ms/step - loss: 2.6592 - sparse_rmse: 0.9134 - val_loss: 2.6190 - val_sparse_rmse: 0.9551
Epoch 4/100
35/35 [==============================] - 4s 107ms/step - loss: 2.3975 - sparse_rmse: 0.8962 - val_loss: 2.3920 - val_sparse_rmse: 0.9480
Epoch 5/100
35/35 [==============================] - 3s 98ms/step - loss: 2.1804 - sparse_rmse: 0.8864 - val_loss: 2.1954 - val_sparse_rmse: 0.9416
Epoch 6/100
35/35 [==============================] - 3s 98ms/step - loss: 1.9823 - sparse_rmse: 0.8716 - val_loss: 2.0263 - val_sparse_rmse: 0.9363
Epoch 7/100
35/35 [==============================] - 4s 103ms/step - loss: 1.8244 - sparse_rmse: 0.8661 - val_

KeyboardInterrupt: 